# The Picture Frame Problem - Formulation 3

A company produces two types of picture frames: each type-1 frame uses 2 hours of labor and 1 ounce of metal and brings $2.25 in profit, and each type-2 frame uses 1 hour of labor and 2 ounces of metal and brings $2.60 in
profit. Each week 4000 labor hours and 5000 ounces of metal are available and the company aims to maximize its weekly profit with an optimal production plan.

To model this problem, let $x_1$ denote the number of type-1 frames produced and $x_2$ denote the number of type-2 frames produced. Then, this can be formulated as the following LP:

\begin{align*}
\underset{x}{\max} \quad &z = 2.25x_1 + 2.6 x_2 \\
\text{s.t.} \ \ \quad &2x_1 + x_2 \leq 4000, \\
&x_1 + 2x_2 \leq 5000, \\
&x \geq 0.
\end{align*}

Let's model this problem using JuMP.

### Problem Data

In [53]:
#Types of picture frames produced
types = [:type_1, :type_2]

#Resources involved
resources = [:labor, :metal]

#Profit made from each type
profit = Dict(zip(types, [2.25,2.6]))

#Quantity available of each resource
quant_avail = Dict(zip(resources, [4000, 5000]));

#If you haven't installed this before
#using Pkg
#Pkg.add("NamedArrays")
using NamedArrays
# recipes (type, resource)
recipe_mat = [ 2 1; 1 2 ]
recipe = NamedArray( recipe_mat, (resources,types), ("resource","type") )


2×2 Named Matrix{Int64}
resource ╲ type │ :type_1  :type_2
────────────────┼─────────────────
:labor          │       2        1
:metal          │       1        2

In [54]:
#Import JuMP package to build an optimization model
using JuMP
#Import HiGHS solver
using HiGHS

#Create a JuMP model named picframe1 that will be solved using the HiGHS solver
picframe3 = Model(HiGHS.Optimizer);

#Add the variables frame[:type_1] and frame[:type_2] that are nonnegative
@variable(picframe3, frame[types] >= 0);

#Create the constraints, name them constraint1 and constraint2
@constraint(picframe3, constraint[i in resources], sum(recipe[i,t]*frame[t] for t in types) <= quant_avail[i]);

@expression(picframe3, objfun, sum(profit[i]*frame[i] for i in types))

#Create our objective function and set it for minimization
@objective(picframe3, Max, objfun);

print(picframe3)

In [55]:
#Solve the model
optimize!(picframe3);
#Outputs detailed information about the solution process
@show solution_summary(picframe3);

Presolving model
2 rows, 2 cols, 4 nonzeros
2 rows, 2 cols, 4 nonzeros
Presolve : Reductions: rows 2(-0); columns 2(-0); elements 4(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.8499947627e+00 Ph1: 2(6); Du: 2(4.84999) 0s
          2    -7.4500000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  7.4500000000e+03
HiGHS run time      :          0.00
solution_summary(picframe3) = * Solver : HiGHS

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution
  Objective value      : 7.45000e+03
  Objective bound      : 7.45000e+03
  Relative gap         : Inf
  Dual objective value : 7.45000e+03

* Work counters
  Solve time (sec)   : 2.00295e-03
  Simplex ite

In [56]:
#Status of primal problem
@show primal_status(picframe3);
#Status of dual problem
@show dual_status(picframe3);
#Final objective value
@show objective_value(picframe3);

println(value.(frame))
#Shadow price of constraints
println(shadow_price.(constraint))

primal_status(picframe3) = MathOptInterface.FEASIBLE_POINT
dual_status(picframe3) = MathOptInterface.FEASIBLE_POINT
objective_value(picframe3) = 7450.0
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:type_1, :type_2]
And data, a 2-element Vector{Float64}:
 1000.0
 2000.0
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:labor, :metal]
And data, a 2-element Vector{Float64}:
 0.6333333333333333
 0.9833333333333334


Suppose that the company decides to produce a third type of picture frame. It takes 2 hours of labor and 3 ounces of metal to produce and generates a profit of $4.5 dollars. To update our model, we just need to change the data!

In [57]:
#Types of picture frames produced
types = [:type_1, :type_2, :type_3]

#Profit made from each type
profit = Dict(zip(types, [2.25,2.6,4.5]))

#If you haven't installed this before
#using Pkg
#Pkg.add("NamedArrays")
using NamedArrays
# recipes (type, resource)
recipe_mat = [ 2 1 2; 1 2 3]
recipe = NamedArray( recipe_mat, (resources,types), ("resource","type") )


2×3 Named Matrix{Int64}
resource ╲ type │ :type_1  :type_2  :type_3
────────────────┼──────────────────────────
:labor          │       2        1        2
:metal          │       1        2        3

In [58]:
picframeupdated = Model(HiGHS.Optimizer);

#Add the variables frame[:type_1] and frame[:type_2] that are nonnegative
@variable(picframeupdated, frame[types] >= 0);

#Create the constraints, name them constraint1 and constraint2
@constraint(picframeupdated, constraint[i in resources], sum(recipe[i,t]*frame[t] for t in types) <= quant_avail[i]);

#Create our objective function and set it for minimization
@objective(picframeupdated, Max, sum(profit[i]*frame[i] for i in types));

print(picframeupdated)

In [59]:
#Solve the model
optimize!(picframeupdated);
#Outputs detailed information about the solution process
@show solution_summary(picframeupdated);

Presolving model
2 rows, 3 cols, 6 nonzeros
2 rows, 3 cols, 6 nonzeros
Presolve : Reductions: rows 2(-0); columns 3(-0); elements 6(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -9.3499894416e+00 Ph1: 2(11); Du: 3(9.34999) 0s
          2    -7.8750000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  7.8750000000e+03
HiGHS run time      :          0.00
solution_summary(picframeupdated) = * Solver : HiGHS

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution
  Objective value      : 7.87500e+03
  Objective bound      : 7.87500e+03
  Relative gap         : Inf
  Dual objective value : 7.87500e+03

* Work counters
  Solve time (sec)   : 9.88960e-04
  Simp

In [60]:
#Final objective value
@show objective_value(picframeupdated);

println(value.(frame))
#Shadow price of constraints
println(shadow_price.(constraint))

objective_value(picframeupdated) = 7875.0
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:type_1, :type_2, :type_3]
And data, a 3-element Vector{Float64}:
  500.0
    0.0
 1500.0
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:labor, :metal]
And data, a 2-element Vector{Float64}:
 0.5625
 1.125
